In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv
/kaggle/input/amexfeather/test_data_f32.ftr
/kaggle/input/amexfeather/train_data.ftr
/kaggle/input/amexfeather/train_data_f32.ftr
/kaggle/input/amexfeather/test_data.ftr


In [21]:
number_of_transactions = 2
data = pd.read_feather('../input/amexfeather/train_data.ftr').groupby('customer_ID').tail(number_of_transactions).set_index('customer_ID', drop=True).sort_index()
labels = pd.read_csv("../input/amex-default-prediction/train_labels.csv").set_index('customer_ID', drop=True).sort_index()
test_df = pd.read_feather('../input/amexfeather/test_data.ftr').groupby('customer_ID').tail(number_of_transactions).set_index('customer_ID', drop=True).sort_index()

**Merge Data**

In [22]:
train_df = pd.merge(data, labels, left_index=True, right_index=True) 
print(train_df)
train_df = train_df.drop(["S_2"],axis=1)
test_df = test_df.drop(["S_2"],axis=1)

                                                          S_2       P_2  \
customer_ID                                                               
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fb... 2018-02-21  0.937500   
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fb... 2018-03-13  0.934570   
00000fd6641609c6ece5454664794f0340ad84dddce9a26... 2018-02-06  0.878906   
00000fd6641609c6ece5454664794f0340ad84dddce9a26... 2018-03-25  0.880371   
00001b22f846c82c51f6e3958ccd81970162bae8b007e80... 2018-02-09  0.866211   
...                                                       ...       ...   
ffff9984b999fccb2b6127635ed0736dda94e544e67e026... 2018-03-07  0.800293   
ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf388145... 2018-02-15  0.754395   
ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf388145... 2018-03-23  0.753906   
fffff1d38b785cef84adeace64f8f83db3a0c31e8d92eab... 2018-02-06  0.969727   
fffff1d38b785cef84adeace64f8f83db3a0c31e8d92eab... 2018-03-14  0.981934   

                        

**Preprocessing Data**

In [23]:
#Dropping the Transaction Dates
#drop_cols = ['S_2'] 
#train_df.drop(drop_cols, inplace=True, axis=1)

print(train_df.columns)
y = train_df.iloc[:, -1].values


def preprocessing_data(data_x):
    
    # convert categorical variables
    data_x1 = pd.get_dummies(data_x)
    print(data_x1.columns)
    
    print(data_x1)

    #Handling missing values
    col_names = data_x1.columns
    imputer = SimpleImputer()
    data_x1 = pd.DataFrame(imputer.fit_transform(data_x1))  
    data_x1.columns = col_names
    
    # Standardization
    scaler = StandardScaler()
    data_x1 = pd.DataFrame(scaler.fit_transform(data_x1), index=data_x1.index, columns=data_x1.columns)
    return data_x1


x_temp = train_df.iloc[:, :-1]
x=preprocessing_data(x_temp)
test_x = preprocessing_data(test_df)

# Splitting into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.2, random_state = 0)

Index(['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42',
       'D_43',
       ...
       'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144', 'D_145',
       'target_x', 'target_y'],
      dtype='object', length=190)
Index(['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42',
       'D_43',
       ...
       'D_117_2.0', 'D_117_3.0', 'D_117_4.0', 'D_117_5.0', 'D_117_6.0',
       'D_120_0.0', 'D_120_1.0', 'D_126_-1.0', 'D_126_0.0', 'D_126_1.0'],
      dtype='object', length=224)
                                                         P_2      D_39  \
customer_ID                                                              
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fb...  0.937500  0.002424   
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fb...  0.934570  0.009117   
00000fd6641609c6ece5454664794f0340ad84dddce9a26...  0.878906  0.536133   
00000fd6641609c6ece5454664794f0340ad84dddce9a26...  0.880371  0.178101   
00001b22f846c82c51f6e3958ccd81970

/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


ValueError: Length mismatch: Expected axis has 55 elements, new values have 220 elements

**Training the Logistic Regression model on the Training set**

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic_classifier = LogisticRegression(random_state = 0)
logistic_classifier.fit(x_train, y_train)

In [ ]:
y_pred_LR = logistic_classifier.predict(x_val)
print(np.concatenate((y_pred_LR.reshape(len(y_pred_LR),1), y_val.reshape(len(y_val),1)),1))

**Making the Confusion Matrix of Logistic Regression model**

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm_LR = confusion_matrix(y_val, y_pred_LR)
print(cm_LR)
accuracy_score(y_val, y_pred_LR)

**Training the KNN model on the Training set**

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier
#kNNclassifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
#kNNclassifier.fit(x_train, y_train)

In [ ]:

#y_pred_KNN = kNNclassifier.predict(x_val)
#print(y)
#print(y_pred_KNN)
#print(y_val)
#print(np.concatenate((y_pred_KNN.reshape(len(y_pred_KNN),1), y_val.reshape(len(y_val),1)),1))

**Making the Confusion Matrix of KNN model**

In [ ]:
#from sklearn.metrics import confusion_matrix, accuracy_score
#cm_KNN = confusion_matrix(y_val, y_pred_KNN)
#print(cm_KNN)
#accuracy_score(y_val, y_pred_KNN)

**Training the SVM model on the Training set**

In [ ]:
#from sklearn.svm import SVC
#svmclassifier = SVC(kernel = 'linear', random_state = 0)
#svmclassifier.fit(x_train, y_train)

In [ ]:
#y_pred_svm = svmclassifier.predict(x_val)
#print(np.concatenate((y_pred_svm.reshape(len(y_pred_svm),1), y_val.reshape(len(y_val),1)),1))

**Making the Confusion Matrix of SVM model**

In [ ]:
#from sklearn.metrics import confusion_matrix, accuracy_score
#cm_svm = confusion_matrix(y_val, y_pred_svm)
#print(cm_svm)
#accuracy_score(y_val, y_pred_svm)

**Training XGBoost on the Training set**

In [ ]:
from xgboost import XGBClassifier
xgb_classifier = XGBClassifier()
xgb_classifier.fit(x_train, y_train)

In [ ]:
y_pred_xgb = xgb_classifier.predict(x_val)
print(np.concatenate((y_pred_xgb.reshape(len(y_pred_xgb),1), y_val.reshape(len(y_val),1)),1))

**Making the Confusion Matrix of XGB**

In [ ]:
cm_XGB = confusion_matrix(y_val, y_pred_xgb)
print(cm_XGB)
accuracy_score(y_val, y_pred_xgb)

**Handling Test Data**

In [ ]:
# Loading dataset test_data.csv
#test = pd.read_csv('../input/amex-default-prediction/test_data.csv').groupby('customer_ID').tail(1).set_index('customer_ID')

#test = pd.read_feather('../input/amexfeather/test_data.ftr').groupby('customer_ID').tail(number_of_transactions).set_index('customer_ID', drop=True).sort_index()

#test = pd.read_csv('../input/amex-default-prediction/test_data.csv').groupby('customer_ID').tail(number_of_transactions).set_index('customer_ID', drop=True).sort_index()
print(test_x)

In [ ]:
predicted_test_data = xgb_classifier.predict(test_x)
print(predicted_test_data)

In [ ]:
output = pd.DataFrame({'customer_ID': test_x.index,'prediction': predicted_test_data})
output.to_csv('submission.csv', index=False, header=True)